In [2]:
import cv2
import numpy as np

def track_and_annotate_balls(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Video cannot be opened.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    first_frame = True
    tracked_balls = []
    video_writers = {}
    paths = {}
    last_positions = {}
    collisions = {i: 0 for i in range(1, 8)}  # Initialize collision counts for each ball

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower_red1 = np.array([0, 130, 176])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160, 140, 159])
        upper_red2 = np.array([180, 255, 255])
        mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)
        red_mask = cv2.bitwise_or(mask1, mask2)

        kernel = np.ones((3,3), np.uint8)
        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)

        red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        red_contours = sorted(red_contours, key=cv2.contourArea, reverse=True)[:7]

        current_balls = []

        for contour in red_contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if radius > 0.4:
                current_balls.append((x, y, radius))

        if first_frame:
            for index, (x, y, radius) in enumerate(current_balls):
                ball_id = index + 1
                tracked_balls.append((x, y, radius, ball_id))
                video_writers[ball_id] = cv2.VideoWriter(f"{output_dir}/ball_{ball_id}.avi", fourcc, fps, (frame_width, frame_height))
                paths[ball_id] = [(int(x), int(y))]
                last_positions[ball_id] = (x, y)
            first_frame = False
        else:
            new_tracked_balls = []
            for (x, y, radius) in current_balls:
                distances = [(i, np.sqrt((x - bx)**2 + (y - by)**2)) for i, (bx, by, br, bi) in enumerate(tracked_balls)]
                closest_ball_index, _ = min(distances, key=lambda x: x[1])
                bx, by, br, bi = tracked_balls.pop(closest_ball_index)
                new_tracked_balls.append((x, y, radius, bi))
                paths[bi].append((int(x), int(y)))

                # Check if the ball has moved significantly
                last_x, last_y = last_positions[bi]
                if np.sqrt((x - last_x)**2 + (y - last_y)**2) > 4.5:
                    # Check for collisions with other balls
                    for _, (other_x, other_y, other_r, other_id) in enumerate(tracked_balls):
                        distance = np.sqrt((x - other_x)**2 + (y - other_y)**2)
                        collision_distance = (radius + other_r) * 1.47
                        if distance < collision_distance:
                            collisions[bi] += 1
                            collisions[other_id] += 1
                            break

            tracked_balls = new_tracked_balls + tracked_balls

        # Update last positions
        for (x, y, _, bi) in tracked_balls:
            last_positions[bi] = (x, y)

        # Draw and record paths
        for (x, y, radius, bi) in tracked_balls:
            highlight_frame = frame.copy()
            for j in range(1, len(paths[bi])):
                cv2.line(highlight_frame, paths[bi][j - 1], paths[bi][j], (255, 0, 0), 2)
            cv2.circle(highlight_frame, (int(x), int(y)), int(radius), (0, 255, 0), 2)
            cv2.putText(highlight_frame, f"{bi}", (int(x - radius/2), int(y + radius/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.putText(highlight_frame, f"Collision: {collisions[bi]}", (10, 30 * bi), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            video_writers[bi].write(highlight_frame)

    cap.release()
    for writer in video_writers.values():
        writer.release()
    cv2.destroyAllWindows()
    print("Video processing complete. Each ball's movement path and collisions have been recorded.")

input_video_path = 'C:/Users/acer/Downloads/video.avi'
output_directory = 'C:/Users/acer/Downloads/output_videos'

track_and_annotate_balls(input_video_path, output_directory)


Video processing complete. Each ball's movement path and collisions have been recorded.
